# Hand Pose and Music Generation

In [1]:
import utils as u 

In [ ]:
x_data, y_data = u.load_data() 
print(y_data)

In [ ]:
x_data, y_data = u.shuffle_data(x_data, y_data, random_seed=20) 
print(y_data)

In [ ]:
split_idx = int(len(x_data) * 0.8)
x_train, y_train, x_test, y_test = u.split_data(x_data, y_data, split_idx)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = u.preprocess_data(x_train, 
                                                                 y_train, 
                                                                 x_test, 
                                                                 y_test)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
print(len(x_train.shape))
print(x_train.max())
print(x_train.min())

print(len(x_test.shape))
print(x_test.max())
print(x_test.min())

In [ ]:
u.explore_data(x_train, y_train, y_test, y_val)

In [ ]:
model = u.build_model()


In [ ]:
history = u.train_model(model, x_train, y_train, x_val, y_val, 
                      epochs=2, batch_size=32)

In [ ]:
u.plot_loss(history)

In [ ]:
test_acc, y_pred = u.test_model(model, x_test, y_test)
print(y_pred)

In [ ]:
model.save('./Handpose-Recognition.h5')

In [ ]:
print(test_acc)

In [ ]:
model.summary()

In [ ]:
!tensorflowjs_converter \
    --input_format=keras \
    --output_format=tfjs_graph_model \
    --saved_model_tags=serve \
    /Users/qianruzhang/Documents/GitHub/Machine-Learning-ML5/Week5-Real-Time-Handpose-Recognition/ML/Handpose-Recognition.keras \
    /Users/qianruzhang/Documents/GitHub/Machine-Learning-ML5/Week5-Real-Time-Posture-Recognition/tfjsmodel
